# Introduction of 3D Spatial Computation Library

This notebook demonstrates the basic functionality of the text-based 3D spatial computation library. It shows how to:
- Initialize a machine with available blocks
- Explore the available tools and their documentation
- Build a simple car structure using wooden blocks
- Handle attachment operations and save the machine to file
- Reuse saved machines as sub-structures in assembly

The test creates a small car frame with vertical structure and suspension components.
The visualization is for collider boxes instead of the block mesh, providing a method to debug the collider settings.

## 1. Build:
### 1.1 Initialize ``Machine()``:
- Start with initialize a ``Machine``
- Review the tool descriptions, the tools are provided with the doc strings to the Agent
- Review the Block descriptions, the available blocks are added to the Agent system prompt

In [ ]:
import os
from spatial.build import Machine
from spatial.utils import machine_preview

from config import SavedMachines

machine = Machine(name="small_car_hm", save_dir=SavedMachines)
print(machine.blocks_storage())
for tool_group in machine.tools:
    print(f"tool_group: {tool_group}")
    for tool in machine.tools[tool_group]:
        print(tool.__name__)
        print(tool.__doc__)
        print("-"*100)

AvailableBlocks = machine.blocks_storage()
print(AvailableBlocks)

save_dir = os.path.join(os.path.dirname(machine.db_path), "machine", machine.name)
os.makedirs(save_dir, exist_ok=True)

### 1.2 Start building:
- ``Machine.start()`` is a tool only provided in the ``build`` stage:

In [ ]:
machine.start(init_shift=[0, 0, 0], init_rotation=[0, 0, 0])

### 1.3 Build by attaching new blocks:
- ``Machine.attach_block_to()`` is the most important and the most frequently used tool, Agent executes the building process by assigning the ``base_block``, ``face``, ``new_block``, and ``Optional[note]`` to the tool:

In [ ]:
# Vertical frame
machine.attach_block_to(base_block=1, face="E", new_block="Small Wooden Block")
machine.attach_block_to(base_block=1, face="F", new_block="Small Wooden Block")
machine.attach_block_to(base_block=2, face="C", new_block="Small Wooden Block", note="Top of the frame")
# Suspensions
machine.attach_block_to(base_block=2, face="A", new_block="Small Wooden Block")
machine.attach_block_to(base_block=2, face="B", new_block="Small Wooden Block")
machine.attach_block_to(base_block=3, face="A", new_block="Small Wooden Block")
machine.attach_block_to(base_block=3, face="B", new_block="Small Wooden Block")

# Further extension of the frame
machine.attach_block_to(base_block=5, face="B", new_block="Small Wooden Block")
machine.attach_block_to(base_block=6, face="A", new_block="Small Wooden Block")
machine.attach_block_to(base_block=7, face="B", new_block="Small Wooden Block")
machine.attach_block_to(base_block=8, face="A", new_block="Small Wooden Block")

scene = machine_preview(machine)
scene.show()

- All building operations including ``Machine.attach_block_to()``, ``Machine.twist_block()``, and ``Machine.shift_block()`` check the validity of the input arguments, in case of: 
    - ``bask_block`` not found
    - ``face`` occupied or not existing
    - ``new_block`` not available
    - Blocks overlap

In [ ]:
# Extra step to test the failure logging
# Face occupied
machine.attach_block_to(base_block=8, face="A", new_block="Small Wooden Block")
# Base block not found
machine.attach_block_to(base_block=42, face="A", new_block="Small Wooden Block")
# New block not available
machine.attach_block_to(base_block=8, face="A", new_block="Cannon")
# Block overlap
machine.attach_block_to(base_block=1, face="C", new_block="Powered Wheel")
machine.to_file(save_dir)

In [ ]:
# Wheels
machine.attach_block_to(base_block=9, face="A", new_block="Powered Wheel")
machine.attach_block_to(base_block=10, face="B", new_block="Powered Wheel")
machine.attach_block_to(base_block=11, face="A", new_block="Powered Wheel")
machine.attach_block_to(base_block=12, face="B", new_block="Powered Wheel")
# Flip half of the wheels
machine.flip_spin(14)
machine.flip_spin(16)
machine.to_file(save_dir)

scene = machine_preview(machine)
scene.show()


### 1.4 Making connections:
- ``Machine.connect_blocks()`` is responsible for connecting two attachable faces (or "sticky" face in code) of existing blocks with a ``Connector()``(child class of ``Block()``) instance
- ``Connector()`` does not have a collider box (according to the game setting).

In [ ]:
machine.connect_blocks(block_a=3, face_a='D', block_b=4, face_b='E', connector='Winch')
machine.connect_blocks(block_a=9, face_a='C', block_b=10, face_b='C', connector='Brace')
machine.connect_blocks(block_a=11, face_a='D', block_b=12, face_b='D', connector='Brace')
machine.connect_blocks(block_a=11, face_a='D', block_b=9, face_b='C', connector='Brace')
machine.connect_blocks(block_a=12, face_a='D', block_b=10, face_b='C', connector='Brace')
machine.connect_blocks(block_a=15, face_a='E', block_b=16, face_b='E', connector='Brace')
machine.connect_blocks(block_a=13, face_a='E', block_b=14, face_b='E', connector='Brace')

In [ ]:
# Add the water cannon and torch
machine.attach_block_to(base_block=4, face="E", new_block="Water Cannon")
machine.twist_block(24, 180)
machine.attach_block_to(base_block=4, face="D", new_block="Torch")

- All building operations lead to modification of block position and orientation, including ``Machine.attach_block_to()``, ``Machine.twist_block()``, and ``Machine.shift_block()``, have internal collision checking mechanism to make sure the collider boxes of the blocks do not overlap.

In [ ]:
machine.shift_block(24, [0, 0, -1])

### 1.5 Control:
- ``Machine()`` provides a series of tools for open-loop controlling:

In [ ]:
print(machine.review_control_config())

In [ ]:
machine.change_control_key(16, 'spinning_forward', 'Alpha2')
machine.change_control_key(15, 'spinning_forward', 'Alpha2')

In [ ]:
machine.add_control_sequence(time=0, key='Alpha2', hold_for=5)
machine.add_control_sequence(time=5, key='UpArrow', hold_for=3)
machine.add_control_sequence(time=10, key='DownArrow', hold_for=5)

### 1.6 Review the Machine state:
- ``Machine.prompt`` property dynamically update the state of the machine as feedback to the Agent

In [ ]:
machine.update_prompt(complete=True, return_summary=True)
print(machine.prompt)

### 1.6 Save and Load the building trajectory:
- ``Machine()`` saves the latest building operation history excluding all filed operations for loading the operation history file back into a ``Machine()`` instance.
- It also records another full operation history for analysis.

In [ ]:
print("operation_history:")
for op in machine.operation_history:
    print(op)

print("operation_history_full:")
for op in machine.operation_history_full:
    print(op)

In [ ]:
machine.rebuild_from_history()

scene = machine_preview(machine)
scene.show()

### 1.7 Save the ``Machine()`` as ``BSG`` file:
- ``Machine.to_file()`` saves the machine into game-ready ``BSG`` file.
- The control sequence will be saved in the ``BSG`` file using ``Lua`` script MOD

In [ ]:
machine.to_file(output_dir=save_dir)

### 1.8 Load a Machine: 
- ``Machine.from_file()`` utilizes ``Machine.rebuild_from_history()`` after machine has been saved to reload a operation history ``JSON`` file back to instance:

In [ ]:
name = "small_car_hm"
machine.from_file(file_path=os.path.join(save_dir, f"{name}.json"))
scene = machine_preview(machine)
scene.show()

### 1.9 Global adjustment:
- ``Machine.rotate()`` and ``Machine.shift()`` are not exposed to Agents as tools in the ``build`` stage, here is to verify the operation on the colliders:

In [ ]:
machine.rotate(0, 0, 70)
scene = machine_preview(machine)
scene.show()

In [ ]:
machine.shift([-5, 0, 0])
scene = machine_preview(machine)
scene.show()

## 2. Assemble:
### 2.1 Initialize the ``Assembly()``：
- ``Assembly()`` is a child class of ``Machine()``, inherits all functions and tools from ``Machine()``
- Special tools exclusively used in ``assemble`` stage allow using multiple saved machine as sub-structures in the entire structure

In [ ]:
from spatial.build import Assembly
from spatial.utils import machine_preview

from config import SavedMachines

assembly = Assembly(name="assemble", save_dir=SavedMachines, db_path="./datacache/default/machine")
for tool_group in assembly.tools:
    print(tool_group)
    for tool in assembly.tools[tool_group]:
        print(tool.__name__)
        print(tool.__doc__)
        print("-"*100)

assembly.reset()

### 2.2 Add a ``Machine()``：
- ``Assembly.add_machine()`` adds a saved machine as a sub-structure

In [ ]:
assembly.add_machine(machine_id="small_car_hm", init_shift=[-5, 0, 0], init_rotation=[0, 0, 0])
assembly.add_machine(machine_id="small_car_hm", init_shift=[5, 0, 0], init_rotation=[0, 0, 0])
assembly.shift_machine("A", [5, 0, 0])
assembly.rotate_machine("A", [0, 0, -45])
assembly.rotate_machine("B", [0, 0, 45])

### 2.3 Making modifications:
- Building operations in the ``build`` stage also work in the ``assemble`` stage

In [ ]:
assembly.connect_blocks(block_a='A_2', face_a='C', block_b='B_2', face_b='C', connector='Brace')
assembly.attach_block_to(base_block='A_3', face='D', new_block='Small Wooden Block')
assembly.attach_block_to(base_block='B_3', face='D', new_block='Small Wooden Block')

### 2.4 Review the Assembly:
- ``Assembly.prompt`` property dynamically update the state of the assembly as feedback to the Agent

In [ ]:
assembly.update_prompt(complete=True, return_summary=True)
print(assembly.prompt)

In [ ]:
assembly.rebuild_from_history()
scene = machine_preview(assembly)
scene.show()

### 2.5 Save the ``Assembly()``:
- ``Assembly()`` has the same saving  mechanism as ``Machine()``

In [ ]:
assembly.to_file(output_dir=f"./datacache/default/machine/{assembly.name}")

### 2.6 Load the ``Assembly()``:
- ``Assembly()`` has the same loading mechanism as ``Machine()``

In [ ]:
assembly.from_file(file_path="datacache/default/machine/assemble/assemble.json")
scene = machine_preview(assembly)
scene.show()

### 2.7 Reuse the ``Assembly()`` as sub-structures:
- ``Assembly()`` can also be reused as sub-structures mixed with ``Machine()``

In [ ]:
assembly = Assembly(name="assemble_2", save_dir=SavedMachines, db_path="./datacache/default/machine")

assembly.add_machine(machine_id="assemble", init_shift=[0, 0, 0], init_rotation=[0, 0, 0])
assembly.add_machine(machine_id="assemble", init_shift=[0, 8, 0], init_rotation=[0, 0, 0])
assembly.add_machine(machine_id="small_car_hm", init_shift=[2.5, -2, 4], init_rotation=[0, 0, 0])

scene = machine_preview(assembly)
scene.show()

In [ ]:
assembly.to_file(output_dir="datacache/default/machine/assemble_2")

- And of course an ``Assembly()`` that has multiple ``Assembly()`` and ``Machine()`` can be reused for further combination as well, recreating the ``copy`` mechanism of the game

In [ ]:
assembly = Assembly(name="assemble_3", save_dir=SavedMachines, db_path="./datacache/default/machine")

assembly.add_machine(machine_id="assemble_2", init_shift=[0, 0, 0], init_rotation=[0, 0, 0])
assembly.add_machine(machine_id="small_car_hm", init_shift=[2.5, -2, -4], init_rotation=[0, 0, 0])

scene = machine_preview(assembly)
scene.show()

In [ ]:
assembly.to_file(output_dir="datacache/default/machine/assemble_3")